# Mapping warehouse v2 GTFS data to ITP IDs

Laurie / November 2022

Investigate:
* How many datasets map to multiple organizations? 
* How many organizations map to multiple datasets?
* How many ITP IDs are assigned in v1 and not assigned in v2?
* Coverage in v2 vs. v1: cases where v1 has all 4 feed types but v2 does not.
* How many datasets end up with a different ITP ID in v2 than in v1?

Overall goal: **How best to assign ITP IDs to GTFS datasets in the v2 pipeline?**

In [ ]:
import pandas as pd
import pandas_gbq
import base64
import datetime


In [ ]:
feeds_raw  = pd.read_gbq(
    """
    select 
        key AS dt_feed_key,
        feed_key AS schedule_feed_key,
        gtfs_dataset_key,
        null AS schedule_to_use_for_rt_validation_gtfs_dataset_key
    from `cal-itp-data-infra-staging`.`laurie_mart_gtfs`.`fct_daily_schedule_feeds`
    where date = '2022-10-31'
    UNION ALL
    select 
        key AS dt_feed_key,
        schedule_feed_key,
        gtfs_dataset_key,
        schedule_to_use_for_rt_validation_gtfs_dataset_key
    from `cal-itp-data-infra-staging`.`laurie_mart_gtfs`.`fct_daily_rt_feed_files`
    where date = '2022-10-31'
    """, project_id = 'cal-itp-data-infra-staging')

In [ ]:
gtfs_datasets_raw  = pd.read_gbq(
    """
    select *
    from `cal-itp-data-infra`.`mart_transit_database`.`dim_gtfs_datasets`
    """, project_id = 'cal-itp-data-infra')

In [ ]:
gtfs_service_data_raw  = pd.read_gbq(
    """
    select *
    from `cal-itp-data-infra`.`mart_transit_database`.`dim_gtfs_service_data`
    """, project_id = 'cal-itp-data-infra')

In [ ]:
services_raw  = pd.read_gbq(
    """
    select *
    from `cal-itp-data-infra`.`mart_transit_database`.`dim_services`
    """, project_id = 'cal-itp-data-infra')

In [ ]:
organizations_raw  = pd.read_gbq(
    """
    select *
    from `cal-itp-data-infra`.`mart_transit_database`.`dim_organizations`
    """, project_id = 'cal-itp-data-infra')

In [ ]:
service_organization_bridge_raw  = pd.read_gbq(
    """
    select *
    from `cal-itp-data-infra`.`mart_transit_database`.`bridge_organizations_x_services_managed`
    """, project_id = 'cal-itp-data-infra')

In [ ]:
old_itp_mapping_raw  = pd.read_gbq(
    """
    select t1.*,
    t2.calitp_itp_id,
    t2.calitp_url_number,
    t2.calitp_feed_name
    from `cal-itp-data-infra-staging.views.gtfs_schedule_fact_daily_feeds` t1
    left join `cal-itp-data-infra-staging.views.gtfs_schedule_dim_feeds` t2
        using(feed_key)
    where date = '2022-10-31'
    """, project_id = 'cal-itp-data-infra')

In [ ]:
# copy for manipulation
feeds = feeds_raw.copy()
gtfs_datasets = gtfs_datasets_raw.copy()
gtfs_service_data = gtfs_service_data_raw.copy()
services = services_raw.copy()
organizations = organizations_raw.copy()
service_organization_bridge = service_organization_bridge_raw.copy()

In [ ]:
# map feeds to airtable datasets
feeds_to_datasets = feeds.merge(gtfs_datasets[['name', 'key', 'base64_url', 'data']], how = 'left', right_on = 'key', left_on = 'gtfs_dataset_key', suffixes = ['_feed', '_dataset'])

In [ ]:
# do the naive thing: dataset --> service --> organization
feeds_to_services = feeds_to_datasets.merge(
    gtfs_service_data[['service_key', 'gtfs_dataset_key', 'category', 'reference_static_gtfs_service_data_key']],
    how = 'left', on = 'gtfs_dataset_key', suffixes = ['', '_gsd'])

In [ ]:
feeds_to_organizations = feeds_to_services.merge(service_organization_bridge, how = 'left', on = 'service_key', suffixes = ['', '_sobridge']).merge(organizations[['key', 'itp_id']], how = 'left', left_on = 'organization_key', right_on = 'key', suffixes = ['', '_org'])

In [ ]:
feeds_to_organizations.loc[feeds_to_organizations.category != "primary", "itp_id"] = None

In [ ]:
# look for one feed with multiple itp ids
itp_ids_per_feed = feeds_to_organizations[feeds_to_organizations.category == "primary"].groupby(by = ['dt_feed_key', 'name'])['itp_id'].nunique().reset_index()

In [ ]:
multiple_itp_ids_per_feed = itp_ids_per_feed[itp_ids_per_feed.itp_id > 1].copy()
multiple_itp_ids_per_feed['feed_with_multiple_itp_ids'] = True

In [ ]:
# now the other direction: ITP IDs that end up with multiple datasets
feeds_per_itp_id = feeds_to_organizations[feeds_to_organizations.category == "primary"].groupby(by = ['itp_id', 'data'])['dt_feed_key'].nunique().reset_index()

In [ ]:
multiple_feeds_per_itp_id = feeds_per_itp_id[feeds_per_itp_id.dt_feed_key > 1].copy()
multiple_feeds_per_itp_id['itp_id_with_multiple_feeds'] = True

In [ ]:
# add flags to the core table
feeds_to_organizations = feeds_to_organizations.merge(multiple_itp_ids_per_feed[['dt_feed_key','feed_with_multiple_itp_ids']], how = 'left', on = 'dt_feed_key')
feeds_to_organizations['feed_with_multiple_itp_ids'].fillna(False, inplace=True)

feeds_to_organizations = feeds_to_organizations.merge(multiple_feeds_per_itp_id[['itp_id', 'data','itp_id_with_multiple_feeds']], how = 'left', on = ['itp_id', 'data'])
feeds_to_organizations['itp_id_with_multiple_feeds'].fillna(False, inplace=True)

In [ ]:
# fill in key for schedule so we can do some grouping by associated feeds
feeds_to_organizations.loc[feeds_to_organizations.data == 'GTFS Schedule', 'schedule_to_use_for_rt_validation_gtfs_dataset_key'] = feeds_to_organizations.loc[feeds_to_organizations.data == 'GTFS Schedule', 'gtfs_dataset_key']

In [ ]:
feeds_to_organizations = feeds_to_organizations.merge(feeds_to_organizations[['gtfs_dataset_key', 'name']].drop_duplicates(), how = 'left',
                                                     left_on = 'schedule_to_use_for_rt_validation_gtfs_dataset_key', right_on = 'gtfs_dataset_key', suffixes = ['', '_ref_sched'])

In [ ]:
feeds_to_organizations_map = feeds_to_organizations[['itp_id', 'name_ref_sched', 'category', 'data','base64_url']].drop_duplicates()

In [ ]:
feeds_to_organizations_map

TODO:

- Will need to manually construct quartets through joins -- can't use pandas pivot because there are dups on `'itp_id', 'name_ref_sched', 'category'`
- Make a wide version which can be compared with v1 -- try joining on all columns then look for discrepancies... 

In [ ]:
feeds_to_organizations_itp_map = feeds_to_organizations_itp_map.pi

In [ ]:
feeds_to_organizations_dedupe = feeds_to_organizations[['name', 'category', '']]

In [ ]:
# make long so each url on own row
old_itp_mapping = old_itp_mapping_raw.melt(id_vars = ['calitp_itp_id', 'calitp_url_number', 'calitp_feed_name'], value_vars = ['raw_gtfs_schedule_url', 'raw_gtfs_rt_vehicle_positions_url', 'raw_gtfs_rt_service_alerts_url',
       'raw_gtfs_rt_trip_updates_url'], var_name = 'type', value_name = 'url')

In [ ]:
# filter out nonexistent datasets, like rows that don't have RT
old_itp_mapping = old_itp_mapping[old_itp_mapping.url.notnull()].copy()

In [ ]:
# have to replace urls with templating to allow for joins
old_itp_mapping['url_to_join'] = old_itp_mapping.url.copy()
old_itp_mapping['url_to_join'] = old_itp_mapping['url_to_join'].str.replace('api_key={{ MTC_511_API_KEY}}','',regex=False)
old_itp_mapping['url_to__join'] = old_itp_mapping['url_to_join'].str.replace('https://api.511.org/transit/servicealerts?','https://api.511.org/transit/servicealerts',regex=False)
old_itp_mapping['url_to_join'] = old_itp_mapping['url_to_join'].str.replace('?&','?',regex=False)
old_itp_mapping['url_to_join'] = old_itp_mapping['url_to_join'].str.replace('?token={{ AC_TRANSIT_API_KEY }}','',regex=False)
old_itp_mapping['url_to_join'] = old_itp_mapping['url_to_join'].str.replace('?apiKey={{ TORRANCE_TRANSIT_API_KEY }}','',regex=False)
old_itp_mapping['url_to_join'] = old_itp_mapping['url_to_join'].str.replace('?key={{ SD_MTS_VP_TU_API_KEY}}','',regex=False)
old_itp_mapping['url_to_join'] = old_itp_mapping['url_to_join'].str.replace('?apiKey={{ SD_MTS_SA_API_KEY}}','',regex=False)
old_itp_mapping['url_to_join'] = old_itp_mapping['url_to_join'].str.replace('?apiKey={{ CULVER_CITY_API_KEY }}','',regex=False)

In [ ]:
data_type_map= {'raw_gtfs_schedule_url': 'GTFS Schedule',
                'raw_gtfs_rt_vehicle_positions_url': 'GTFS VehiclePositions',
                'raw_gtfs_rt_service_alerts_url': 'GTFS Alerts',
                'raw_gtfs_rt_trip_updates_url': 'GTFS TripUpdates'}

In [ ]:
old_itp_mapping['type'] = old_itp_mapping.type.map(data_type_map)

In [ ]:
# convert to float for join (comes in as int)
old_itp_mapping.calitp_itp_id = old_itp_mapping.calitp_itp_id.astype(float)

In [ ]:
# core columns; drop duplicates because if there are multiple services but they all map to same organization, that isn't actually a problem 
v2_map = feeds_to_organizations[['dt_feed_key','name', 'base64_url', 'organization_name','data','category', 'itp_id', 'feed_with_multiple_itp_ids', 'itp_id_with_multiple_feeds']].drop_duplicates()

In [ ]:
v2_map['decoded_url'] = v2_map.base64_url.apply(lambda x: base64.urlsafe_b64decode(x).decode())

In [ ]:
# how many cases where same schedule is used for validation by multiple sets of rt?
feeds_to_organizations.groupby(by = 'reference_static_gtfs_service_data_key')['dt_feed_key'].nunique().reset_index().sort_values(by = 'dt_feed_key', ascending = False)

In [ ]:
feeds_to_organizations[feeds_to_organizations.reference_static_gtfs_service_data_key.isin(['recEvfRIonwtIzAJI', 'recdmcXbBgn9jFe6u'])]

In [ ]:
compare = v2_map.merge(old_itp_mapping, how = 'outer', left_on = ['decoded_url', 'itp_id'], right_on = ['url_to_join', 'calitp_itp_id'])

In [ ]:
compare.rename(columns = {'itp_id': 'v2_itp_id', 'calitp_itp_id': 'v1_itp_id', 'name': 'gtfs_dataset_name'},inplace = True)

In [ ]:
itp_mapping = compare[['v2_itp_id', 'v1_itp_id']].drop_duplicates()

In [ ]:
old_to_new = itp_mapping.groupby('v1_itp_id')['v2_itp_id'].nunique().reset_index().rename(columns = {'v2_itp_id': 'ct_v2_itp_ids'})

In [ ]:
compare = compare.merge(old_to_new, how = 'left', on = 'v1_itp_id')

In [ ]:
new_to_old = itp_mapping.groupby('v2_itp_id')['v1_itp_id'].nunique().reset_index().rename(columns = {'v1_itp_id': 'ct_v1_itp_ids'})

In [ ]:
compare = compare.merge(new_to_old, how = 'left', on = 'v2_itp_id')

In [ ]:
compare.to_csv('compare_itp_id_assignments.csv', index = False)

In [ ]:
len(compare[compare.v2_itp_id.notnull() & compare.v1_itp_id.notnull()])

In [ ]:
len(compare[(compare.v2_itp_id.isnull() | compare.v1_itp_id.isnull()) & ((compare.category == "primary") | (compare.calitp_url_number == 0))])

In [ ]:
compare[(compare.v2_itp_id.isnull() | compare.v1_itp_id.isnull()) & ((compare.category == "primary") | (compare.category.isnull()))]

In [ ]:
compare.to_csv('compare_itp_id_assignments.csv', index = False)